In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Rice.zip

unzip:  cannot find or open /content/drive/MyDrive/Rice.zip, /content/drive/MyDrive/Rice.zip.zip or /content/drive/MyDrive/Rice.zip.ZIP.


# Mục mới



---



---


**PHẦN 1: TRAIN MODEL**


---



---



**1) Khai báo thư viện**

In [3]:
!pip install keras
!pip install sklearn
!pip install tensorflow
!pip install optimizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for optimizers: filename=Optimizers-2.1-py3-none-any.whl size=2284 sha256=3c323d90c73d159e520e570ca5da3ac0fa0de6059b53efedc2c297d388cb3a72
  Stored in directory: /root/.cache/pip/wheels/56/a5/4d/f679a391b5fca0b18c5e2fcd66ebff8900d97d6d95713915b9
Successfully built optimizers
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [4]:
#Khai bao thu vien
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers.core import Dropout
from imutils import paths
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from google.colab.patches import cv2_imshow
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import os

ImportError: ignored

**2) Khởi tạo model**

*2.1) DenseNet*

In [ ]:
#Model_DenseNet121
from tensorflow.keras.applications import DenseNet121
baseModel = DenseNet121(weights="imagenet", include_top=False, input_tensor=Input(shape=(64, 64, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2,2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
print(model.summary())

*2.2) VGG16*

In [ ]:
# # Model VGG16
# from tensorflow.keras.applications import VGG16
# baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(64, 64, 3)))
# headModel = baseModel.output
# headModel = AveragePooling2D(pool_size=(2,2))(headModel)
# headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(3, activation="softmax")(headModel)
# model = Model(inputs=baseModel.input, outputs=headModel)
# print(model.summary())

*2.3) MobileNet*

In [ ]:
# #Model MobileNet
# from tensorflow.keras.applications import MobileNet
# baseModel = MobileNet(weights="imagenet", include_top=False, input_tensor=Input(shape=(64, 64, 3)))
# headModel = baseModel.output
# headModel = AveragePooling2D(pool_size=(2,2))(headModel)
# headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(3, activation="softmax")(headModel)
# model = Model(inputs=baseModel.input, outputs=headModel)
# print(model.summary())

*2.4) InceptionV3*

In [ ]:
# #Model Inception
# from tensorflow.keras.applications import InceptionV3
# baseModel = InceptionV3(weights="imagenet", include_top=False, input_tensor=Input(shape=(76, 76, 3)))
# headModel = baseModel.output
# headModel = AveragePooling2D(pool_size=(1,1))(headModel)
# headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(1024, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(3, activation="softmax")(headModel)
# model = Model(inputs=baseModel.input, outputs=headModel)
# print(model.summary())

**3) Thiết lập LR, Batch_size, Epochs**

In [ ]:
#thiet lap thong so lr, batch_size va epochs
INIT_LR = 1e-4
BS = 8
EPOCHS = 20

**4) Khởi tạo ảnh đầu vào**

In [ ]:
#khoi tao input
imagePaths = list(paths.list_images('/content/Rot_Img/train'))
data = []
labels = []

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (64,64))
    # cv2_imshow(image)
    data.append(image)
    labels.append(label)
data = np.array(data, dtype="float") / 255.0

**5) Mã hoá one-hot nhãn**

In [ ]:
#one-hot label
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 3)

**6) Thiết lập dữ liệu train**

In [ ]:
#set du lieu train
(X_train, X_val, y_train, y_val) = train_test_split(data, labels, test_size=0.1, random_state=42)

**7) Compile và train model**

In [ ]:
#compile + train
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
H = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=BS, steps_per_epoch=len(X_train) // BS, epochs=EPOCHS)

**8) Đồ thị**

In [ ]:
#do thi
import matplotlib.pyplot as plt

plt.plot(np.arange(0, EPOCHS), H.history['loss'], label='loss')
plt.plot(np.arange(0, EPOCHS), H.history['accuracy'], label = 'accuracy')
plt.plot(np.arange(0, EPOCHS), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, EPOCHS), H.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend()
plt.show()

**9) Lưu model**

In [ ]:
#save model
model.save('XLA_ModelDenseNet121.h5')

**HẾT PHẦN 1**

---



---



***Load model đã train trước đó***

In [ ]:
from keras.models import load_model

model1 = load_model('/content/drive/MyDrive/ImgProc/XLA_DenseNet121.h5')



---



---

**PHẦN 2: TEST VỚI DỮ LIỆU TRONG DATASET BAN ĐẦU**


---



---



**1) Khai báo thư viện**

In [ ]:
#Khai bao thu vien
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers.core import Dropout
from imutils import paths
from google.colab.patches import cv2_imshow
from keras.models import load_model
import numpy as np
import pickle
import cv2
import os

**2) Khởi tạo dữ liệu test + One-hot nhãn**

In [ ]:
#Khoi tao du lieu test
imagePaths_test = list(paths.list_images('/content/Rot_Img/test'))
data_test = []
labels_test = []
for path_test in imagePaths_test:
    label2 = path_test.split(os.path.sep)[-2]
    image2 = cv2.imread(path_test)
    image2 = cv2.resize(image2, (64,64))
    data_test.append(image2)
    labels_test.append(label2)
data_test = np.array(data_test, dtype="float") / 255.0

#one-hot label
le_test = LabelEncoder()
labels_test = le_test.fit_transform(labels_test)
labels_test = np_utils.to_categorical(labels_test, 3)

**3) Thiết lập dữ liệu test**

In [ ]:
#set du lieu test
X_test = data_test
y_test = labels_test

**4) Đánh giá tệp test**

In [ ]:
score_test = model1.evaluate(X_test, y_test)
print('Score test = ', score_test)

**6) Bảng thống kê các giá trị phân loại**


> Accuracy = (TP+TN)/(TP+FP+FN+TN)

> Precision = TP/(TP+FP)

> Recall = TP/(TP+FN)

> F1 Score = 2(Recall * Precision) / (Recall + Precision)

















In [ ]:
y_pred_test = model1.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=le_test.classes_, digits = 5))

In [ ]:
print("So sanh test")
testY = np.argmax(y_test, axis=1).ravel()
predtestY = np.argmax(y_pred_test, axis=1).ravel()
# print(np.argmax(y_test, axis = 1))
print(testY)
print(predtestY)

In [ ]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(testY, predtestY)
print(cnf_matrix)

In [ ]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims = True)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.5f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot non-normalized confusion matrix
class_names = [0, 1, 2]
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

**HẾT PHẦN 2**


---



---





---



---


**PHẦN 3: TEST DỮ LIỆU VỚI MỘT SỐ ẢNH LẤY TỪ BÊN NGOÀI DATASET**


---



---



**1) Kiểm tra với các hình ảnh bên ngoài tập dataset ban đầu**

In [ ]:
def define_label(x):
  if x == 0:
    return print('Bacterial Leaf Blight')
  if x == 1:
    return print('Brown spot')
  if x == 2:
    return print('Leaf smut')

In [ ]:
img = cv2.imread('/content/drive/MyDrive/ImgProc/ls4.png')
img = cv2.resize(img,(64,64))
cv2_imshow(img)
img = img.reshape(1,64,64,3)
data_leaf = np.array(img, dtype = 'float') /255.0
one = model1.predict(data_leaf)
print(one)
x = np.argmax(one)
print(x)
print(define_label(x))

NameError: ignored

In [ ]:
img1 = cv2.imread('/content/drive/MyDrive/ImgProc/Ba1.jpg')
img1 = cv2.resize(img1,(64,64))
cv2_imshow(img1)
img1 = img1.reshape(1,64,64,3)
data_leaf1 = np.array(img1, dtype = 'float') /255.0
one1 = model1.predict(data_leaf1)
print(one1)
x1 = np.argmax(one1)
print(x1)
print(define_label(x1))

NameError: ignored

In [ ]:
img2 = cv2.imread('/content/drive/MyDrive/ImgProc/br5.png')
img2 = cv2.resize(img2,(64,64))
cv2_imshow(img2)
img2 = img2.reshape(1,64,64,3)
data_leaf2 = np.array(img2, dtype = 'float') /255.0
one2 = model1.predict(data_leaf2)
print(one2)
x2 = np.argmax(one2)
print(x2)
print(define_label(x2))